In [1]:
%pwd

'e:\\Projects\\Unfinished\\Medical Chatbot\\research'

In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
def load_pdf(data):
    loader = DirectoryLoader(data, 
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    return loader.load()

extracted_data = load_pdf(r'E:\Projects\Unfinished\Medical Chatbot\Data')

Now we need to make the chunks

In [4]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

text_chunks=text_split(extracted_data)
print("Len of Text Chunks: ", len(text_chunks))

Len of Text Chunks:  20990


In [5]:
from langchain.embeddings import HuggingFaceEmbeddings

#384 dimensional DB -> need to know this for pinecone

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [6]:
embeddings = download_hugging_face_embeddings()

C:\Users\Ahmad\AppData\Local\Temp\ipykernel_8508\897455035.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [8]:
query_result = embeddings.embed_query("Hello World")
print("Length of Query Result: ", len(query_result))

Length of Query Result:  384


Pinecone Database!

In [17]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path=r'E:\Projects\Unfinished\Medical Chatbot\.env.local')

api_key_pinecone = os.getenv('API_KEY_PINECONE')
api_key_google = os.getenv('API_KEY_GOOGLE')

In [19]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=api_key_pinecone)

index_name = "medbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

Store chunks here

In [20]:
import os
os.environ["PINECONE_API_KEY"] = api_key_pinecone
os.environ["GOOGLE_API_KEY"] = api_key_google

In [21]:
#Embed each chunk and upsert to Pinecone
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)

In [22]:
# Existing index 
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)


In [23]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})
retrieved_docs = retriever.invoke("What is Acne?")
retrieved_docs

[Document(id='7099049d-1687-4ce0-b785-6cc1b3bca61a', metadata={'page': 55.0, 'page_label': '26', 'source': 'E:\\Projects\\Unfinished\\Medical Chatbot\\Data\\MedicalBook.pdf'}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(id='9adb5f0b-5b51-451b-b99e-c6604e0d1a3d', metadata={'page': 54.0, 'page_label': '25', 'source': 'E:\\Projects\\Unfinished\\Medical Chatbot\\Data\\MedicalBook.pdf'}, page_content='Pathological Stage and Recurrence in Radical\nProstatectomy Cases.’’Journal of Urology (March\n1998): 935-940.\nNancy J. Nordenson\nAcid reflux see Heartburn\nAcidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when\nthe pores of the skin become clogged with oil, dead\nskin cells, and bacteria.\nDescription\nAcne vulgaris, the medical term for common acne,\nis the most common skin diseas

Initialize the Model!

In [26]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.4,
    max_tokens=500
)

In [28]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer"
    "the user's question. If you do not know the answer, say that you do not know"
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [29]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [31]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])

Acne is a common skin disease characterized by pimples on the face, chest, and back.  It occurs when pores become clogged with oil, dead skin cells, and bacteria.  Acne vulgaris is the medical term for common acne, and it's the most common skin disease.


In [33]:
response = rag_chain.invoke({"input": "What is Digital Signal Processing?"})
print(response["answer"])

This text discusses digital x-rays and ultrasound technology, but it does not define Digital Signal Processing. Therefore, I cannot answer your question using the provided context. 
